In [67]:

import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict
import re
from typing import Any, Dict, List, Union
# !pip install xlsxwriter
import xlsxwriter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)


#Rough Flattened Structure

In [68]:
def flatten_with_array_handling_safe(data: Union[Dict, List], parent_key: str = '', sep: str = '_') -> Dict[str, Any]:
    items = []
    #These secions are list of disctionaries and need to be preserved as jsons for each id. 
    #To be flattened later in the pipeline.
    preserve_as_json = [
        'telephonehistory', 
        'employmenthistory', 
        'enquiryhistorytop',
        'creditagreementsummary',
        'accountmonthlypaymenthistory',
        'identificationhistory'
    ]
    try:
        if isinstance(data, dict):
            for key, value in data.items():
                new_key = f"{parent_key}{sep}{key}" if parent_key else key
                if value is None:
                    items.append((new_key, None))
                elif key in preserve_as_json and isinstance(value, list):
                    items.append((new_key, json.dumps(value)))
                elif isinstance(value, (dict, list)):
                    nested_result = flatten_with_array_handling_safe(value, new_key, sep)
                    items.extend(nested_result.items())
                else:
                    items.append((new_key, value))
        elif isinstance(data, list):
            for i, item in enumerate(data):
                new_key = f"{parent_key}{sep}{i}" if parent_key else str(i)
                if item is None:
                    items.append((new_key, None))
                elif isinstance(item, (dict, list)):
                    nested_result = flatten_with_array_handling_safe(item, new_key, sep)
                    items.extend(nested_result.items())
                else:
                    items.append((new_key, item))
        else:
            if parent_key:
                items.append((parent_key, data))
    except Exception as e:
        print(f"Error processing key '{parent_key}': {e}")
        print(f"Data type: {type(data)}, Data: {str(data)[:100]}...")
        return {}
    return dict(items)

In [69]:
#credit report json flattening script
json_file_path = "credit_report_sample.json"

with open(json_file_path, 'r') as file:
    credit_reports = json.load(file)

print(f"Loaded {len(credit_reports)} credit reports")

flattened_reports = []
for i, report in enumerate(credit_reports):
    print(f"Processing report {i+1}/{len(credit_reports)}")
    try:
        flattened_report = flatten_with_array_handling_safe(report)
        flattened_reports.append(flattened_report)
        print(f"Successfully processed report {i+1}")
    except Exception as e:
        print(f"Error processing report {i+1}: {e}")
        continue

if flattened_reports:
    df_flattened = pd.DataFrame(flattened_reports)
    print(f"Flattened data shape: {df_flattened.shape}")
    print(f"Number of features after flattening: {df_flattened.shape[1]}")
    df_flattened.to_csv("output/v0_flattened_credit_reports.csv", index=False)
else:
    print("No reports were successfully processed")

Loaded 3 credit reports
Processing report 1/3
Successfully processed report 1
Processing report 2/3
Successfully processed report 2
Processing report 3/3
Successfully processed report 3
Flattened data shape: (3, 133)
Number of features after flattening: 133


In [70]:
#This is the intermediate output of the flattening process. 
# To check if files are parsed properly  
#note sections like 'telephonehistory', 'employmenthistory', etc. are preserved as jsons for each id.
print("Intermediate Flattened DataFrame:")
df_flattened.head()

Intermediate Flattened DataFrame:


,application_id,data_consumerfullcredit_subjectlist_reference,data_consumerfullcredit_subjectlist_consumerid,data_consumerfullcredit_subjectlist_searchoutput,data_consumerfullcredit_accountrating_noofotheraccountsbad,data_consumerfullcredit_accountrating_noofotheraccountsgood,data_consumerfullcredit_accountrating_noofretailaccountsbad,data_consumerfullcredit_accountrating_noofretailaccountsgood,data_consumerfullcredit_accountrating_nooftelecomaccountsbad,data_consumerfullcredit_accountrating_noofautoloanaccountsbad,data_consumerfullcredit_accountrating_noofautoloanccountsgood,data_consumerfullcredit_accountrating_noofhomeloanaccountsbad,data_consumerfullcredit_accountrating_nooftelecomaccountsgood,data_consumerfullcredit_accountrating_noofhomeloanaccountsgood,data_consumerfullcredit_accountrating_noofjointloanaccountsbad,data_consumerfullcredit_accountrating_noofstudyloanaccountsbad,data_consumerfullcredit_accountrating_noofcreditcardaccountsbad,data_consumerfullcredit_accountrating_noofjointloanaccountsgood,data_consumerfullcredit_accountrating_noofstudyloanaccountsgood,data_consumerfullcredit_accountrating_noofcreditcardaccountsgood,data_consumerfullcredit_accountrating_noofpersonalloanaccountsbad,data_consumerfullcredit_accountrating_noofpersonalloanaccountsgood,data_consumerfullcredit_enquirydetails_productid,data_consumerfullcredit_enquirydetails_matchingrate,data_consumerfullcredit_enquirydetails_subscriberenquiryengineid,data_consumerfullcredit_enquirydetails_subscriberenquiryresultid,data_consumerfullcredit_guarantorcount_accounts,data_consumerfullcredit_guarantorcount_guarantorssecured,data_consumerfullcredit_guarantordetails_guarantorgender,data_consumerfullcredit_guarantordetails_guarantorotherid,data_consumerfullcredit_guarantordetails_guarantoraddress1,data_consumerfullcredit_guarantordetails_guarantoraddress2,data_consumerfullcredit_guarantordetails_guarantoraddress3,data_consumerfullcredit_guarantordetails_guarantorpassport,data_consumerfullcredit_guarantordetails_guarantorfirstname,data_consumerfullcredit_guarantordetails_guarantorothername,data_consumerfullcredit_guarantordetails_guarantordateofbirth,data_consumerfullcredit_guarantordetails_guarantornationalidno,data_consumerfullcredit_guarantordetails_guarantorhometelephone,data_consumerfullcredit_guarantordetails_guarantorworktelephone,data_consumerfullcredit_guarantordetails_guarantordriverlicenceno,data_consumerfullcredit_guarantordetails_guarantormobiletelephone,data_consumerfullcredit_telephonehistory,data_consumerfullcredit_employmenthistory,data_consumerfullcredit_enquiryhistorytop,data_consumerfullcredit_creditaccountsummary_rating,data_consumerfullcredit_creditaccountsummary_amountarrear,data_consumerfullcredit_creditaccountsummary_amountarrear1,data_consumerfullcredit_creditaccountsummary_totalaccounts,data_consumerfullcredit_creditaccountsummary_totalaccounts1,data_consumerfullcredit_creditaccountsummary_lastjudgementdate,data_consumerfullcredit_creditaccountsummary_lastjudgementdate1,data_consumerfullcredit_creditaccountsummary_totalaccountarrear,data_consumerfullcredit_creditaccountsummary_totalaccountarrear1,data_consumerfullcredit_creditaccountsummary_totaljudgementamount,data_consumerfullcredit_creditaccountsummary_totaloutstandingdebt,data_consumerfullcredit_creditaccountsummary_totaljudgementamount1,data_consumerfullcredit_creditaccountsummary_totaloutstandingdebt1,data_consumerfullcredit_creditaccountsummary_totaldishonouredamount,data_consumerfullcredit_creditaccountsummary_totalmonthlyinstalment,data_consumerfullcredit_creditaccountsummary_totalnumberofjudgement,data_consumerfullcredit_creditaccountsummary_totaldishonouredamount1,data_consumerfullcredit_creditaccountsummary_totalmonthlyinstalment1,data_consumerfullcredit_creditaccountsummary_totalnumberofjudgement1,data_consumerfullcredit_creditaccountsummary_totalnumberofdishonoured,data_consumerfullcredit_creditaccountsummary_totalnumberofdishonoured1,data_consumerfullcredit_creditaccou

In [71]:
# grabs all keys from nested dicts/lists, goes deep up to max_depth
def get_all_keys(data, prefix="", max_depth=10, current_depth=0):
    keys = set()
    if current_depth > max_depth:
        return keys
    try:
        if isinstance(data, dict):
            for key, value in data.items():
                full_key = f"{prefix}.{key}" if prefix else key
                keys.add(full_key)
                if isinstance(value, (dict, list)) and value:
                    keys.update(get_all_keys(value, full_key, max_depth, current_depth + 1))
        elif isinstance(data, list):
            for index, item in enumerate(data):
                item_prefix = f"{prefix}[{index}]" if prefix else f"[{index}]"
                keys.update(get_all_keys(item, item_prefix, max_depth, current_depth + 1))
    except Exception as e:
        print(f"Error at key '{prefix}': {e}")
    return keys

all_keys = set()
for i, report in enumerate(credit_reports):
    try:
        print(f"Processing report {i+1}/{len(credit_reports)}")
        if 'data' in report:
            all_keys.update(get_all_keys(report['data'], 'data'))
        else:
            all_keys.update(get_all_keys(report))
    except Exception as e:
        print(f"Error processing report {i+1}: {e}")
        continue

sorted_keys = sorted(all_keys)
sorted_keys  

# save sorted_keys as a DataFrame and export to CSV
df_keys = pd.DataFrame(sorted_keys, columns=['key'])
df_keys.to_csv('output/sorted_keys.csv', index=False)

# split dot-separated key into columns
max_parts = df_keys['key'].str.count(r'\.').max() + 1
df_keys_split = df_keys['key'].str.split('.', n=int(max_parts), expand=True)
df_keys_split.columns = [f'level_{i+1}' for i in range(df_keys_split.shape[1])]
df_keys_split['full_key'] = df_keys['key']
df_keys_split.to_csv('output/sorted_keys_split.csv', index=False)

# print(f"\nGrouped by main sections:")
# sections = {}
# for key in sorted_keys:
#     parts = key.split('.')
#     main_section = parts[1] if len(parts) > 1 else key
#     if main_section not in sections:
#         sections[main_section] = []
#     sections[main_section].append(key)

# for section, keys in sections.items():
#     print(f"\n{section}:")
#     for key in keys:
#         print(f"  - {key}")

Processing report 1/3
Processing report 2/3
Processing report 3/3


#Save Individual Child Tables in Credit Inquiry Report as Flattened Tables

In [72]:
#Extract individual tables from credit reports to excel files
def extract_individual_tables(credit_reports):
    """
    Extract each individual table/section from credit reports into separate DataFrames
    """
    tables = {}
    
    def extract_table_data(data, table_name, parent_id=None):
        """Helper function to extract data for a specific table"""
        table_data = []
        
        if isinstance(data, list):
            # If it's a list, each item becomes a row
            for i, item in enumerate(data):
                if isinstance(item, dict):
                    row = item.copy()
                    if parent_id is not None:
                        row['parent_id'] = parent_id
                    row['record_index'] = i
                    table_data.append(row)
                else:
                    # Handle primitive values in lists
                    row = {'value': item, 'record_index': i}
                    if parent_id is not None:
                        row['parent_id'] = parent_id
                    table_data.append(row)
        elif isinstance(data, dict):
            # If it's a dict, it becomes a single row
            row = data.copy()
            if parent_id is not None:
                row['parent_id'] = parent_id
            table_data.append(row)
        
        return table_data
    
    def process_nested_structure(data, prefix="", parent_id=None):
        """Recursively process nested structure to extract tables"""
        
        if isinstance(data, dict):
            for key, value in data.items():
                current_path = f"{prefix}.{key}" if prefix else key
                
                if isinstance(value, list) and len(value) > 0:
                    # This is a table (list of records)
                    table_data = extract_table_data(value, current_path, parent_id)
                    if table_data:
                        if current_path not in tables:
                            tables[current_path] = []
                        tables[current_path].extend(table_data)
                
                elif isinstance(value, dict):
                    # Check if this dict contains only primitive values (single record table)
                    has_nested = any(isinstance(v, (dict, list)) for v in value.values())
                    
                    if not has_nested:
                        # This is a single-record table
                        table_data = extract_table_data(value, current_path, parent_id)
                        if table_data:
                            if current_path not in tables:
                                tables[current_path] = []
                            tables[current_path].extend(table_data)
                    else:
                        # Continue recursing
                        process_nested_structure(value, current_path, parent_id)
    
    # Process each credit report
    for report_idx, report in enumerate(credit_reports):
        report_id = f"report_{report_idx}"
        
        if 'data' in report:
            process_nested_structure(report['data'], 'data', report_id)
        else:
            process_nested_structure(report, '', report_id)
    
    # Convert to DataFrames
    dataframes = {}
    for table_name, table_data in tables.items():
        if table_data:
            try:
                df = pd.DataFrame(table_data)
                dataframes[table_name] = df
                print(f"Created table '{table_name}': {df.shape[0]} rows, {df.shape[1]} columns")
            except Exception as e:
                print(f"Error creating DataFrame for '{table_name}': {e}")
    
    return dataframes

def extract_specific_tables(credit_reports):
    """
    Extract specific known tables from credit reports with better structure
    """
    tables = {}
    
    # Define known table structures
    table_definitions = {
        'personal_details': {
            'path': ['data', 'consumerfullcredit', 'personaldetailssummary'],
            'type': 'single_record'
        },
        'account_rating': {
            'path': ['data', 'consumerfullcredit', 'accountrating'],
            'type': 'single_record'
        },
        'credit_account_summary': {
            'path': ['data', 'consumerfullcredit', 'creditaccountsummary'],
            'type': 'single_record'
        },
        'telephone_history': {
            'path': ['data', 'consumerfullcredit', 'telephonehistory'],
            'type': 'array'
        },
        'employment_history': {
            'path': ['data', 'consumerfullcredit', 'employmenthistory'],
            'type': 'array'
        },
        'enquiry_history': {
            'path': ['data', 'consumerfullcredit', 'enquiryhistorytop'],
            'type': 'array'
        },
        'credit_agreements': {
            'path': ['data', 'consumerfullcredit', 'creditagreementsummary'],
            'type': 'array'
        },
        'payment_history': {
            'path': ['data', 'consumerfullcredit', 'accountmonthlypaymenthistory'],
            'type': 'array'
        },
        'delinquency_info': {
            'path': ['data', 'consumerfullcredit', 'deliquencyinformation'],
            'type': 'single_record'
        },
        'guarantor_details': {
            'path': ['data', 'consumerfullcredit', 'guarantordetails'],
            'type': 'single_record'
        }
    }
    
    def get_nested_value(data, path):
        """Navigate nested dictionary using path"""
        current = data
        for key in path:
            if isinstance(current, dict) and key in current:
                current = current[key]
            else:
                return None
        return current
    
    # Extract each table
    for table_name, definition in table_definitions.items():
        table_data = []
        
        for report_idx, report in enumerate(credit_reports):
            report_id = f"report_{report_idx}"
            
            # Get the data at the specified path
            data = get_nested_value(report, definition['path'])
            
            if data is not None:
                if definition['type'] == 'array' and isinstance(data, list):
                    # Handle array data
                    for record_idx, record in enumerate(data):
                        if isinstance(record, dict):
                            row = record.copy()
                            row['report_id'] = report_id
                            row['record_index'] = record_idx
                            table_data.append(row)
                
                elif definition['type'] == 'single_record' and isinstance(data, dict):
                    # Handle single record data
                    row = data.copy()
                    row['report_id'] = report_id
                    table_data.append(row)
        
        # Create DataFrame if we have data
        if table_data:
            try:
                df = pd.DataFrame(table_data)
                tables[table_name] = df
                print(f"Created '{table_name}' table: {df.shape[0]} rows, {df.shape[1]} columns")
            except Exception as e:
                print(f"Error creating '{table_name}' table: {e}")
        else:
            print(f"No data found for '{table_name}' table")
    
    return tables

# Usage example
print("Extracting individual tables from credit reports...")

# Method 1: Automatic extraction of all tables
print("\n=== Method 1: Automatic Table Extraction ===")
all_tables = extract_individual_tables(credit_reports)

print(f"\nFound {len(all_tables)} tables:")
for table_name, df in all_tables.items():
    print(f"  - {table_name}: {df.shape}")

# Method 2: Extract specific known tables
print("\n=== Method 2: Specific Table Extraction ===")
specific_tables = extract_specific_tables(credit_reports)

print(f"\nExtracted {len(specific_tables)} specific tables:")
for table_name, df in specific_tables.items():
    print(f"  - {table_name}: {df.shape}")

# Display sample data from each table
print("\n=== Sample Data from Each Table ===")
for table_name, df in specific_tables.items():
    print(f"\n{table_name.upper()}:")
    print("-" * 50)
    if not df.empty:
        print(df.head(2))
        print(f"Columns: {list(df.columns)}")
    else:
        print("No data available")
def export_credit_report_formatted(tables_dict, filename="credit_report_formatted.xlsx"):
    # dumps each table to a tab in excel, adds some formatting
    try:
        from openpyxl.styles import Font, PatternFill, Alignment
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            for table_name, df in tables_dict.items():
                sheet_name = table_name.replace('.', '_').replace('/', '_')[:31]
                if not df.empty:
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    worksheet = writer.sheets[sheet_name]
                    header_font = Font(bold=True, color="FFFFFF")
                    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
                    for cell in worksheet[1]:
                        cell.font = header_font
                        cell.fill = header_fill
                        cell.alignment = Alignment(horizontal="center")
                    for column in worksheet.columns:
                        max_length = 0
                        column_letter = column[0].column_letter
                        for cell in column:
                            try:
                                if len(str(cell.value)) > max_length:
                                    max_length = len(str(cell.value))
                            except:
                                pass
                        worksheet.column_dimensions[column_letter].width = min(max_length + 2, 50)
                else:
                    pd.DataFrame(columns=['No Data Available']).to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Successfully exported formatted tables to '{filename}'")
    except ImportError:
        print("openpyxl not available for formatting. Please install openpyxl.")
        return False
    except Exception as e:
        print(f"Error exporting formatted Excel: {e}")
        return False
    return True

# Usage:
specific_tables = extract_specific_tables(credit_reports)
export_credit_report_formatted(specific_tables, "output/credit_report_formatted.xlsx")


Extracting individual tables from credit reports...

=== Method 1: Automatic Table Extraction ===
Created table 'data.consumerfullcredit.subjectlist': 3 rows, 4 columns
Created table 'data.consumerfullcredit.accountrating': 3 rows, 19 columns
Created table 'data.consumerfullcredit.enquirydetails': 3 rows, 5 columns
Created table 'data.consumerfullcredit.guarantorcount': 3 rows, 3 columns
Created table 'data.consumerfullcredit.guarantordetails': 3 rows, 15 columns
Created table 'data.consumerfullcredit.telephonehistory': 9 rows, 7 columns
Created table 'data.consumerfullcredit.employmenthistory': 15 rows, 6 columns
Created table 'data.consumerfullcredit.enquiryhistorytop': 35 rows, 7 columns
Created table 'data.consumerfullcredit.creditaccountsummary': 3 rows, 24 columns
Created table 'data.consumerfullcredit.deliquencyinformation': 3 rows, 5 columns
Created table 'data.consumerfullcredit.creditagreementsummary': 27 rows, 19 columns
Created table 'data.consumerfullcredit.personaldetails

True

#Feature Engineering Steps

In [73]:


def create_combined_bureau_features(json_file_path: str) -> pd.DataFrame:
    """
    Combined function bureau features using credit report JSON data. 
    Args:
        json_file_path: Path to the credit report JSON file
        
    Returns:
        DataFrame with comprehensive bureau features based on available data
    """
    
    # Load the JSON data
    with open(json_file_path, 'r') as file:
        credit_reports = json.load(file)
    
    features_list = []
    
    for report_idx, report in enumerate(credit_reports):
        features = {'report_id': report_idx}
        
        # Navigate to the main data section
        if 'data' in report and 'consumerfullcredit' in report['data']:
            data = report['data']['consumerfullcredit']
        else:
            data = {}
        
        # Helper functions
        def safe_int(value, default=0):
            try:
                if value is None or str(value).strip() == '' or str(value) == '-':
                    return default
                return int(float(str(value)))
            except:
                return default
        
        def safe_float(value, default=0.0):
            try:
                if value is None or str(value).strip() == '' or str(value) == '-':
                    return default
                return float(str(value).replace(',', ''))
            except:
                return default
        
        def clean_amount(amount_str):
            if not amount_str or amount_str == '-':
                return 0
            try:
                return float(str(amount_str).replace(',', '').replace('NGN', '').strip())
            except:
                return 0
        
        def safe_json_parse(json_str):
            if isinstance(json_str, str):
                try:
                    return json.loads(json_str)
                except:
                    return []
            elif isinstance(json_str, list):
                return json_str
            else:
                return []
        
        def parse_date(date_str):
            if not date_str or date_str == '-':
                return None
            try:
                if 'T' in str(date_str):
                    return datetime.strptime(str(date_str).split('T')[0], '%Y-%m-%d')
                elif '/' in str(date_str):
                    return datetime.strptime(str(date_str), '%d/%m/%Y')
                else:
                    return datetime.strptime(str(date_str)[:10], '%Y-%m-%d')
            except:
                return None
        
        current_date = datetime.now()
        
        # ===== DEMOGRAPHIC FEATURES  =====
        personal = data.get('personaldetailssummary', {})
        
        # Age features
        birthdate = personal.get('birthdate', '')
        if birthdate and birthdate != '-':
            try:
                if '/' in str(birthdate):
                    birth_year = int(str(birthdate).split('/')[-1])
                else:
                    birth_year = int(str(birthdate)[:4])
                features['age'] = 2024 - birth_year
                features['age_group_young'] = 1 if features['age'] < 25 else 0
                features['age_group_prime'] = 1 if 25 <= features['age'] <= 45 else 0
                features['age_group_mature'] = 1 if 45 < features['age'] <= 65 else 0
                features['age_group_senior'] = 1 if features['age'] > 65 else 0
                features['age_squared'] = features['age'] ** 2
                features['age_risk_factor'] = 1 if features['age'] < 25 or features['age'] > 65 else 0
            except:
                features['age'] = None
                features['age_group_young'] = 0
                features['age_group_prime'] = 0
                features['age_group_mature'] = 0
                features['age_group_senior'] = 0
                features['age_squared'] = 0
                features['age_risk_factor'] = 1
        else:
            features['age'] = None
            features['age_group_young'] = 0
            features['age_group_prime'] = 0
            features['age_group_mature'] = 0
            features['age_group_senior'] = 0
            features['age_squared'] = 0
            features['age_risk_factor'] = 1
        
        # Gender features
        # Gender feature can be later discarded based on regulatory requirement. 
        # Generally Race and Gender are found to be discriminatory features.
        # However, we will keep it for now as it is part of the original data
        gender = personal.get('gender', '')
        gender = str(gender).lower() if gender else ''
        features['gender_male'] = 1 if gender in ['male', 'm'] else 0
        features['gender_female'] = 1 if gender in ['female', 'f'] else 0
        features['gender_unknown'] = 1 if gender == '' else 0
        
        # Contact and verification features
        features['has_email'] = 1 if personal.get('emailaddress') and personal.get('emailaddress') != '-' else 0
        features['has_home_phone'] = 1 if personal.get('hometelephoneno') and personal.get('hometelephoneno') != '-' else 0
        features['has_mobile'] = 1 if personal.get('cellularno') and personal.get('cellularno') != '-' else 0
        features['has_national_id'] = 1 if personal.get('nationalidno') and personal.get('nationalidno') != '-' else 0
        features['has_bvn'] = 1 if personal.get('bankverificationno') and personal.get('bankverificationno') != '-' else 0
        features['has_employer_info'] = 1 if personal.get('employerdetail') and personal.get('employerdetail') != '-' else 0
        
        # Contact Completeness scores
        features['contact_completeness'] = features['has_email'] + features['has_home_phone'] + features['has_mobile']
        features['id_completeness'] = features['has_national_id'] + features['has_bvn']
        features['profile_completeness'] = (features['contact_completeness'] + features['id_completeness'] + 
                                          features['has_employer_info']) / 6
        
        # Dependants
        features['num_dependants'] = safe_int(personal.get('dependants'))
        features['has_dependants'] = 1 if features['num_dependants'] > 0 else 0
        features['high_dependants'] = 1 if features['num_dependants'] > 3 else 0
        
        # ===== ACCOUNT PERFORMANCE FEATURES  =====
        account_rating = data.get('accountrating', {})
        
        # Good accounts by type
        good_accounts = {
            'other': safe_int(account_rating.get('noofotheraccountsgood')),
            'retail': safe_int(account_rating.get('noofretailaccountsgood')),
            'telecom': safe_int(account_rating.get('nooftelecomaccountsgood')),
            'auto': safe_int(account_rating.get('noofautoloanccountsgood')),
            'home': safe_int(account_rating.get('noofhomeloanaccountsgood')),
            'joint': safe_int(account_rating.get('noofjointloanaccountsgood')),
            'study': safe_int(account_rating.get('noofstudyloanaccountsgood')),
            'credit_card': safe_int(account_rating.get('noofcreditcardaccountsgood')),
            'personal': safe_int(account_rating.get('noofpersonalloanaccountsgood'))
        }
        
        # Bad accounts by type
        bad_accounts = {
            'other': safe_int(account_rating.get('noofotheraccountsbad')),
            'retail': safe_int(account_rating.get('noofretailaccountsbad')),
            'telecom': safe_int(account_rating.get('nooftelecomaccountsbad')),
            'auto': safe_int(account_rating.get('noofautoloanaccountsbad')),
            'home': safe_int(account_rating.get('noofhomeloanaccountsbad')),
            'joint': safe_int(account_rating.get('noofjointloanaccountsbad')),
            'study': safe_int(account_rating.get('noofstudyloanaccountsbad')),
            'credit_card': safe_int(account_rating.get('noofcreditcardaccountsbad')),
            'personal': safe_int(account_rating.get('noofpersonalloanaccountsbad'))
        }
        
        # Individual account type features engineered features
        for acc_type in good_accounts.keys():
            features[f'{acc_type}_good_accounts'] = good_accounts[acc_type]
            features[f'{acc_type}_bad_accounts'] = bad_accounts[acc_type]
            features[f'{acc_type}_total_accounts'] = good_accounts[acc_type] + bad_accounts[acc_type]
            features[f'has_{acc_type}_accounts'] = 1 if features[f'{acc_type}_total_accounts'] > 0 else 0
            features[f'{acc_type}_bad_ratio'] = (bad_accounts[acc_type] / 
                                               max(features[f'{acc_type}_total_accounts'], 1))
        
        # Aggregate account metrics
        features['total_good_accounts'] = sum(good_accounts.values())
        features['total_bad_accounts'] = sum(bad_accounts.values())
        features['total_accounts'] = features['total_good_accounts'] + features['total_bad_accounts']
        
        # Account performance ratios
        features['bad_account_ratio'] = features['total_bad_accounts'] / max(features['total_accounts'], 1)
        features['good_account_ratio'] = features['total_good_accounts'] / max(features['total_accounts'], 1)
        features['account_performance_score'] = features['good_account_ratio']
        
        # Account diversity and risk indicators
        features['account_type_diversity'] = sum(1 for acc_type in good_accounts.keys() 
                                               if good_accounts[acc_type] + bad_accounts[acc_type] > 0)
        features['high_risk_account_types'] = (features['has_personal_accounts'] + 
                                             features['has_credit_card_accounts'])
        features['secured_account_types'] = (features['has_auto_accounts'] + 
                                           features['has_home_accounts'])
        
        # ===== CREDIT SUMMARY FEATURES =====
        credit_summary = data.get('creditaccountsummary', {})
        
        # Financial exposure
        features['total_outstanding_debt'] = clean_amount(credit_summary.get('totaloutstandingdebt'))
        features['amount_in_arrears'] = clean_amount(credit_summary.get('amountarrear'))
        features['total_monthly_payment'] = clean_amount(credit_summary.get('totalmonthlyinstalment'))
        features['total_judgement_amount'] = clean_amount(credit_summary.get('totaljudgementamount'))
        features['total_dishonoured_amount'] = clean_amount(credit_summary.get('totaldishonouredamount'))
        
        # Debt ratios and utilization
        features['arrears_to_debt_ratio'] = features['amount_in_arrears'] / max(features['total_outstanding_debt'], 1)
        features['debt_service_ratio'] = features['total_monthly_payment'] / max(features['total_outstanding_debt'] / 12, 1)
        features['judgement_to_debt_ratio'] = features['total_judgement_amount'] / max(features['total_outstanding_debt'], 1)
        
        # Debt amount categories
        features['high_debt'] = 1 if features['total_outstanding_debt'] > 1000000 else 0
        features['medium_debt'] = 1 if 100000 <= features['total_outstanding_debt'] <= 1000000 else 0
        features['low_debt'] = 1 if 0 < features['total_outstanding_debt'] < 100000 else 0
        features['no_debt'] = 1 if features['total_outstanding_debt'] == 0 else 0
        
        # Credit rating features
        features['credit_rating'] = safe_int(credit_summary.get('rating'))
        features['credit_rating_squared'] = features['credit_rating'] ** 2
        features['credit_rating_log'] = np.log1p(features['credit_rating'])
        features['low_risk_rating'] = 1 if features['credit_rating'] <= 10 else 0
        features['medium_risk_rating'] = 1 if 10 < features['credit_rating'] <= 50 else 0
        features['high_risk_rating'] = 1 if 50 < features['credit_rating'] <= 100 else 0
        features['very_high_risk_rating'] = 1 if features['credit_rating'] > 100 else 0
        
        # Account status features
        features['total_active_accounts'] = safe_int(credit_summary.get('totalaccounts'))
        features['accounts_in_arrears'] = safe_int(credit_summary.get('totalaccountarrear'))
        features['accounts_in_good_condition'] = safe_int(credit_summary.get('totalaccountingodcondition'))
        features['total_judgements'] = safe_int(credit_summary.get('totalnumberofjudgement'))
        features['total_dishonoured'] = safe_int(credit_summary.get('totalnumberofdishonoured'))
        
        # Account condition ratios
        features['arrears_account_ratio'] = features['accounts_in_arrears'] / max(features['total_active_accounts'], 1)
        features['good_condition_ratio'] = features['accounts_in_good_condition'] / max(features['total_active_accounts'], 1)
        
        # Severe delinquency indicators
        features['has_judgements'] = 1 if features['total_judgements'] > 0 else 0
        features['has_dishonoured_payments'] = 1 if features['total_dishonoured'] > 0 else 0
        features['has_arrears'] = 1 if features['amount_in_arrears'] > 0 else 0
        features['multiple_judgements'] = 1 if features['total_judgements'] > 1 else 0
        features['chronic_dishonoured'] = 1 if features['total_dishonoured'] > 5 else 0
        
        # ===== DELINQUENCY FEATURES  =====
        delinquency = data.get('deliquencyinformation', {})
        features['months_in_arrears'] = safe_int(delinquency.get('monthsinarrears'))
        features['months_in_arrears_squared'] = features['months_in_arrears'] ** 2
        features['months_in_arrears_log'] = np.log1p(features['months_in_arrears'])
        features['minor_delinquency'] = 1 if 1 <= features['months_in_arrears'] <= 3 else 0
        features['moderate_delinquency'] = 1 if 4 <= features['months_in_arrears'] <= 6 else 0
        features['severe_delinquency'] = 1 if 7 <= features['months_in_arrears'] <= 12 else 0
        features['chronic_delinquency'] = 1 if features['months_in_arrears'] > 12 else 0
        features['delinquency_severity_score'] = min(features['months_in_arrears'] / 24, 1)
        
        # ===== ENQUIRY BEHAVIOR FEATURES  =====
        enquiry_history = safe_json_parse(data.get('enquiryhistorytop', []))
        features['total_enquiries'] = len(enquiry_history)
        
        # Enquiry analysis
        recent_enquiries_30d = 0
        recent_enquiries_90d = 0
        recent_enquiries_180d = 0
        recent_enquiries_365d = 0
        unique_lenders = set()
        enquiry_reasons = []
        
        for enquiry in enquiry_history:
            if isinstance(enquiry, dict):
                # Lender analysis
                lender = enquiry.get('subscribername', '')
                if lender and lender != '-':
                    unique_lenders.add(lender)
                
                # Reason analysis
                reason = enquiry.get('enquiryreason', '')
                if reason and reason != '-':
                    enquiry_reasons.append(reason)
                
                # Time-based analysis
                date_requested = enquiry.get('daterequested', '')
                if date_requested and date_requested != '-':
                    try:
                        enq_date = parse_date(date_requested)
                        if enq_date:
                            days_ago = (current_date - enq_date).days
                            if days_ago <= 30:
                                recent_enquiries_30d += 1
                            if days_ago <= 90:
                                recent_enquiries_90d += 1
                            if days_ago <= 180:
                                recent_enquiries_180d += 1
                            if days_ago <= 365:
                                recent_enquiries_365d += 1
                    except:
                        pass
        
        features['recent_enquiries_30d'] = recent_enquiries_30d
        features['recent_enquiries_90d'] = recent_enquiries_90d
        features['recent_enquiries_180d'] = recent_enquiries_180d
        features['recent_enquiries_365d'] = recent_enquiries_365d
        features['unique_lenders'] = len(unique_lenders)
        features['unique_enquiry_reasons'] = len(set(enquiry_reasons))
        
        # Enquiry momentum (Premier-style features)
        features['enquiries_momentum_30_90d'] = recent_enquiries_30d / max(recent_enquiries_90d - recent_enquiries_30d, 1)
        features['enquiries_momentum_90_180d'] = (recent_enquiries_90d - recent_enquiries_30d) / max(recent_enquiries_180d - recent_enquiries_90d, 1)
        features['enquiries_momentum_180_365d'] = (recent_enquiries_180d - recent_enquiries_90d) / max(recent_enquiries_365d - recent_enquiries_180d, 1)
        
        # Enquiry intensity and behavior
        features['enquiry_intensity_30d'] = recent_enquiries_30d / max(features['total_enquiries'], 1)
        features['enquiry_intensity_90d'] = recent_enquiries_90d / max(features['total_enquiries'], 1)
        features['lender_diversity'] = len(unique_lenders) / max(features['total_enquiries'], 1)
        features['credit_hungry_30d'] = 1 if recent_enquiries_30d > 2 else 0
        features['credit_hungry_90d'] = 1 if recent_enquiries_90d > 3 else 0
        features['excessive_enquiries'] = 1 if features['total_enquiries'] > 10 else 0
        features['enquiry_frequency_score'] = recent_enquiries_90d * 2 + recent_enquiries_180d + recent_enquiries_365d * 0.5
        
        # ===== EMPLOYMENT STABILITY  =====
        employment_history = safe_json_parse(data.get('employmenthistory', []))
        features['employment_records'] = len(employment_history)
        features['employment_stability'] = 1 if features['employment_records'] <= 2 else 0
        features['job_hopper'] = 1 if features['employment_records'] > 5 else 0
        features['employment_diversity'] = len(set(emp.get('occupation', '') for emp in employment_history 
                                                if isinstance(emp, dict) and emp.get('occupation')))
        features['has_employment_history'] = 1 if features['employment_records'] > 0 else 0
        
        # ===== TELEPHONE HISTORY  =====
        telephone_history = safe_json_parse(data.get('telephonehistory', []))
        features['telephone_records'] = len(telephone_history)
        features['telephone_stability'] = 1 if features['telephone_records'] <= 3 else 0
        features['frequent_phone_changes'] = 1 if features['telephone_records'] > 5 else 0
        features['has_telephone_history'] = 1 if features['telephone_records'] > 0 else 0
        features['phone_stability_score'] = 1 / (1 + features['telephone_records'])
        
        # ===== PAYMENT HISTORY ANALYSIS  =====
        payment_history = safe_json_parse(data.get('accountmonthlypaymenthistory', []))
        
        # Analyze 24-month payment patterns
        payment_scores = []
        perfect_payments = 0
        late_payments = 0
        very_late_payments = 0
        
        for account in payment_history:
            if isinstance(account, dict):
                for month in range(1, 25):  # m01 to m24
                    month_key = f'm{month:02d}'
                    if month_key in account:
                        value = account[month_key]
                        if value == '0' or value == 0:
                            payment_scores.append(0)
                            perfect_payments += 1
                        elif value == '#' or value is None or value == '':
                            continue
                        elif str(value).replace('.', '').isdigit():
                            score = min(float(value), 100)
                            payment_scores.append(score)
                            if score > 0:
                                late_payments += 1
                            if score > 30:
                                very_late_payments += 1
        
        total_payment_records = len(payment_scores)
        if total_payment_records > 0:
            features['avg_payment_delay'] = np.mean(payment_scores)
            features['max_payment_delay'] = max(payment_scores)
            features['min_payment_delay'] = min(payment_scores)
            features['std_payment_delay'] = np.std(payment_scores)
            features['payment_consistency'] = max(0, 1 - (np.std(payment_scores) / (np.mean(payment_scores) + 1)))
            features['perfect_payment_ratio'] = perfect_payments / total_payment_records
            features['late_payment_ratio'] = late_payments / total_payment_records
            features['very_late_payment_ratio'] = very_late_payments / total_payment_records
            features['payment_volatility'] = np.std(payment_scores) / (np.mean(payment_scores) + 1)
        else:
            features['avg_payment_delay'] = 0
            features['max_payment_delay'] = 0
            features['min_payment_delay'] = 0
            features['std_payment_delay'] = 0
            features['payment_consistency'] = 1
            features['perfect_payment_ratio'] = 1
            features['late_payment_ratio'] = 0
            features['very_late_payment_ratio'] = 0
            features['payment_volatility'] = 0
        
        features['total_payment_records'] = total_payment_records
        features['has_payment_history'] = 1 if total_payment_records > 0 else 0
        features['extensive_payment_history'] = 1 if total_payment_records > 50 else 0
        
        # Payment trend analysis (recent vs historical)
        recent_payment_scores = payment_scores[:12] if len(payment_scores) >= 12 else payment_scores
        historical_payment_scores = payment_scores[12:] if len(payment_scores) > 12 else []
        
        if recent_payment_scores and historical_payment_scores:
            features['payment_trend'] = np.mean(historical_payment_scores) - np.mean(recent_payment_scores)
        else:
            features['payment_trend'] = 0
        
        features['recent_payment_performance'] = 1 - (np.mean(recent_payment_scores) / 30) if recent_payment_scores else 1
        
        # ===== CREDIT AGREEMENTS ANALYSIS (20 features) =====
        credit_agreements = safe_json_parse(data.get('creditagreementsummary', []))
        features['total_credit_agreements'] = len(credit_agreements)
        
        # Agreement status analysis
        active_agreements = 0
        closed_agreements = 0
        written_off_agreements = 0
        paid_up_agreements = 0
        total_credit_limit = 0
        total_current_balance = 0
        total_overdue_amount = 0
        
        for agreement in credit_agreements:
            if isinstance(agreement, dict):
                status = agreement.get('accountstatus', '')
                status = str(status).lower() if status is not None else ''
                
                if 'open' in status:
                    active_agreements += 1
                elif 'closed' in status:
                    closed_agreements += 1
                elif 'written' in status:
                    written_off_agreements += 1
                elif 'paid' in status:
                    paid_up_agreements += 1
                
                # Financial metrics from agreements
                current_balance = clean_amount(agreement.get('currentbalanceamt', 0))
                opening_balance = clean_amount(agreement.get('openingbalanceamt', 0))
                overdue_amount = clean_amount(agreement.get('amountoverdue', 0))
                
                total_current_balance += current_balance
                total_credit_limit += opening_balance
                total_overdue_amount += overdue_amount
        
        features['active_agreements'] = active_agreements
        features['closed_agreements'] = closed_agreements
        features['written_off_agreements'] = written_off_agreements
        features['paid_up_agreements'] = paid_up_agreements
        features['total_current_balance'] = total_current_balance
        features['total_credit_limit'] = total_credit_limit
        features['total_overdue_amount'] = total_overdue_amount
        
        # Agreement ratios and indicators
        features['active_agreement_ratio'] = active_agreements / max(features['total_credit_agreements'], 1)
        features['written_off_ratio'] = written_off_agreements / max(features['total_credit_agreements'], 1)
        features['credit_utilization'] = total_current_balance / max(total_credit_limit, 1)
        features['overdue_to_balance_ratio'] = total_overdue_amount / max(total_current_balance, 1)
        
        features['has_active_agreements'] = 1 if active_agreements > 0 else 0
        features['has_written_off'] = 1 if written_off_agreements > 0 else 0
        features['multiple_written_off'] = 1 if written_off_agreements > 1 else 0
        features['has_overdue_amounts'] = 1 if total_overdue_amount > 0 else 0
        
        # Credit agreement diversity
        agreement_types = set()
        for agreement in credit_agreements:
            if isinstance(agreement, dict):
                subscriber = agreement.get('subscribername', '').lower()
                if 'bank' in subscriber:
                    agreement_types.add('bank')
                elif 'microfinance' in subscriber:
                    agreement_types.add('microfinance')
                elif 'card' in subscriber:
                    agreement_types.add('credit_card')
                else:
                    agreement_types.add('other')
        
        features['agreement_type_diversity'] = len(agreement_types)
        features['has_bank_agreements'] = 1 if 'bank' in agreement_types else 0
        features['has_microfinance_agreements'] = 1 if 'microfinance' in agreement_types else 0
        
        # ===== GUARANTOR FEATURES  =====
        guarantor_details = data.get('guarantordetails', {})
        guarantor_count = data.get('guarantorcount', {})
        
        features['has_guarantor_info'] = 1 if any(v for v in guarantor_details.values() if v and v != '-') else 0
        features['guarantor_accounts'] = safe_int(guarantor_count.get('accounts'))
        features['guaranteed_secured_accounts'] = safe_int(guarantor_count.get('guarantorssecured'))
        features['is_guarantor'] = 1 if features['guarantor_accounts'] > 0 else 0
        features['guarantor_risk_exposure'] = features['guarantor_accounts'] / max(features['total_accounts'], 1)
        
        # ===== COMPOSITE RISK SCORES  =====
        
        # Payment behavior score (0-1, higher = better)
        features['payment_behavior_score'] = (
            features['perfect_payment_ratio'] * 0.4 +
            features['payment_consistency'] * 0.3 +
            (1 - min(features['avg_payment_delay'] / 30, 1)) * 0.3
        )
        
        # Credit management score (0-1, higher = better)
        features['credit_management_score'] = (
            features['good_account_ratio'] * 0.3 +
            (1 - features['bad_account_ratio']) * 0.3 +
            min(features['account_type_diversity'] / 5, 1) * 0.2 +
            (1 - min(features['credit_utilization'], 1)) * 0.2
        )
        
        # Financial stability score (0-1, higher = better)
        features['financial_stability_score'] = (
            (1 - min(features['arrears_to_debt_ratio'], 1)) * 0.4 +
            (1 - features['has_judgements']) * 0.3 +
            (1 - features['has_dishonoured_payments']) * 0.3
        )
        
        # Profile completeness score (0-1, higher = better)
        features['profile_quality_score'] = (
            features['profile_completeness'] * 0.4 +
            features['employment_stability'] * 0.3 +
            features['telephone_stability'] * 0.3
        )
        
        # Credit seeking behavior score (0-1, higher = more aggressive)
        features['credit_seeking_score'] = (
            min(features['recent_enquiries_90d'] / 5, 1) * 0.4 +
            features['enquiry_intensity_90d'] * 0.3 +
            (1 - min(features['lender_diversity'], 1)) * 0.3
        )
        
        # Delinquency severity score
        features['delinquency_severity_score'] = (
            features['delinquency_severity_score'] * 0.4 +
            features['arrears_to_debt_ratio'] * 0.3 +
            (features['has_judgements'] + features['has_dishonoured_payments']) * 0.15 +
            features['written_off_ratio'] * 0.15
        )
        
        # Overall risk score (0-1, higher = more risky)
        features['overall_risk_score'] = (
            features['bad_account_ratio'] * 0.20 +
            min(features['credit_rating'] / 200, 1) * 0.18 +
            features['arrears_to_debt_ratio'] * 0.15 +
            features['delinquency_severity_score'] * 0.12 +
            (1 - features['payment_behavior_score']) * 0.12 +
            features['credit_seeking_score'] * 0.08 +
            features['written_off_ratio'] * 0.08 +
            (1 - features['financial_stability_score']) * 0.07
        )
        
        # Risk category based on overall score
        if features['overall_risk_score'] < 0.15:
            features['risk_category'] = 'Very Low'
        elif features['overall_risk_score'] < 0.3:
            features['risk_category'] = 'Low'
        elif features['overall_risk_score'] < 0.5:
            features['risk_category'] = 'Medium'
        elif features['overall_risk_score'] < 0.7:
            features['risk_category'] = 'High'
        else:
            features['risk_category'] = 'Very High'
        
        # Additional risk flags
        features['high_risk_flag'] = 1 if features['overall_risk_score'] > 0.5 else 0
        features['very_high_risk_flag'] = 1 if features['overall_risk_score'] > 0.7 else 0
        features['low_risk_flag'] = 1 if features['overall_risk_score'] < 0.3 else 0
        
        # Credit worthiness score (inverse of risk)
        features['creditworthiness_score'] = 1 - features['overall_risk_score']
        
        # Experian-style score (300-850 scale)
        base_score = 300
        score_range = 550
        normalized_score = (1 - features['overall_risk_score']) * score_range + base_score
        features['experian_style_score'] = min(max(normalized_score, 300), 850)
        
        # Risk grade based on score
        if features['experian_style_score'] >= 750:
            features['risk_grade'] = 'A'
        elif features['experian_style_score'] >= 700:
            features['risk_grade'] = 'B'
        elif features['experian_style_score'] >= 650:
            features['risk_grade'] = 'C'
        elif features['experian_style_score'] >= 600:
            features['risk_grade'] = 'D'
        elif features['experian_style_score'] >= 550:
            features['risk_grade'] = 'E'
        else:
            features['risk_grade'] = 'F'
        
        features_list.append(features)
    
    # Convert to DataFrame
    df_features = pd.DataFrame(features_list)
    
    # Fill missing values
    numeric_columns = df_features.select_dtypes(include=[np.number]).columns
    df_features[numeric_columns] = df_features[numeric_columns].fillna(0)
    
    categorical_columns = df_features.select_dtypes(include=['object']).columns
    df_features[categorical_columns] = df_features[categorical_columns].fillna('unknown')
    
    return df_features

# Usage
print("Creating combined bureau features using available data...")
combined_features = create_combined_bureau_features('credit_report_sample.json')
combined_features.to_csv('output/final_bureau_features.csv', index=False)
# print(f"Created {combined_features.shape[1]} features for {combined_features.shape[0]} reports")

# Display feature categories
feature_categories = {
    'Demographic': [col for col in combined_features.columns if any(x in col for x in ['age', 'gender', 'dependant'])],
    'Contact & Verification': [col for col in combined_features.columns if any(x in col for x in ['has_', 'contact', 'id_', 'profile'])],
    'Account Performance': [col for col in combined_features.columns if any(x in col for x in ['account', 'good', 'bad', 'ratio']) and 'payment' not in col],
    'Financial Exposure': [col for col in combined_features.columns if any(x in col for x in ['debt', 'arrears', 'amount', 'balance', 'utilization'])],
    'Credit Rating': [col for col in combined_features.columns if 'rating' in col and 'account' not in col],
    'Delinquency': [col for col in combined_features.columns if any(x in col for x in ['months_in_arrears', 'delinquency', 'judgement', 'dishonoured'])],
    'Enquiry Behavior': [col for col in combined_features.columns if 'enquir' in col],
    'Payment History': [col for col in combined_features.columns if 'payment' in col],
    'Employment & Stability': [col for col in combined_features.columns if any(x in col for x in ['employment', 'telephone', 'stability'])],
    'Credit Agreements': [col for col in combined_features.columns if 'agreement' in col],
    'Guarantor': [col for col in combined_features.columns if 'guarantor' in col],
    'Composite Scores': [col for col in combined_features.columns if 'score' in col],
    'Risk Indicators': [col for col in combined_features.columns if any(x in col for x in ['risk', 'flag', 'creditworth', 'grade'])]
}

print(f"\nFeature breakdown by category:")
total_features = 0
for category, features in feature_categories.items():
    print(f"  {category}: {len(features)} features")
    total_features += len(features)

print(f"\nTotal categorized features: {total_features}")

# Display sample of key features
key_features = ['report_id', 'age', 'total_accounts', 'credit_rating', 'bad_account_ratio', 
                'months_in_arrears', 'recent_enquiries_90d', 'payment_behavior_score', 
                'overall_risk_score', 'experian_style_score', 'risk_grade']

print(f"\nSample of key features:")
print(combined_features[key_features].head())

print(f"\nRisk distribution:")
print(combined_features['risk_category'].value_counts())

print(f"\nRisk grade distribution:")
print(combined_features['risk_grade'].value_counts().sort_index())

Creating combined bureau features using available data...

Feature breakdown by category:
  Demographic: 14 features
  Contact & Verification: 33 features
  Account Performance: 67 features
  Financial Exposure: 22 features
  Credit Rating: 7 features
  Delinquency: 17 features
  Enquiry Behavior: 13 features
  Payment History: 17 features
  Employment & Stability: 9 features
  Credit Agreements: 10 features
  Guarantor: 4 features
  Composite Scores: 12 features
  Risk Indicators: 14 features

Total categorized features: 239

Sample of key features:
   report_id  age  total_accounts  credit_rating  bad_account_ratio  \
0          0   33               7             13           0.000000   
1          1   39              17              2           0.000000   
2          2   41               3            109           0.333333   

   months_in_arrears  recent_enquiries_90d  payment_behavior_score  \
0                 13                     0                0.568000   
1                 